## Serial LETK Filtering


In [ ]:
import Simulator
import Observation
import Statistics

import KalmanFilter
import ETKalmanFilter
import SLETKalmanFilter
import IEWParticleFilter

import Comparer

## Loading simulator 

For Kalman filtering we have to be able to propagate a Gaussian distribution in time and to have access to the linear operators

In [ ]:
timestamp = "2021_08_26-12_22_18"

grid, simulator = Simulator.from_file(timestamp)

## Loading observations 

Observation positions which are fixed for a setup and values from the truth

In [ ]:
obs_timestamp = "2021_08_26-12_22_28"
observation = Observation.from_file(grid, timestamp, obs_timestamp)

In [ ]:
observation.plot_positions()

## Definition of initial distribution 

The inital state is equipped with a Gaussian distribution specified by its mean and covariance. 

In [ ]:
prior_args = Statistics.prior_args_from_file(timestamp)

## New truth if changes intended

In [ ]:
new_truth = False

In [ ]:
if new_truth:
    observation.clear_observations()
    observation.set_regular_positions(10)
    observation.plot_positions()

    statistics_truth = Statistics.Statistics(simulator, 1)
    statistics_truth.set_prior(prior_args)

    for t in range(10):
        statistics_truth.propagate(25)
        statistics_truth.plot()
        observation.observe(statistics_truth.mean)

## Kalman filtering

Updating/conditioning the distributions at every observation time

In [ ]:
statistics_kf = Statistics.Statistics(simulator)
statistics_kf.set_prior(prior_args)

statistics_kf.plot()

In [ ]:
kalmanFilter = KalmanFilter.Kalman(statistics_kf, observation)

for t in range(observation.N_obs):
    print(t)
    statistics_kf.propagate(25)
    kalmanFilter.filter(statistics_kf.mean, statistics_kf.cov, observation.obses[t])
    statistics_kf.plot()

## ETKF filtering

Updating/conditioning the distributions at every observation time

In [ ]:
statistics_etkf = Statistics.Statistics(simulator, 100)
statistics_etkf.set_prior(prior_args)

statistics_etkf.plot()

In [ ]:
etkFilter = ETKalmanFilter.ETKalman(statistics_etkf, observation)

for t in range(observation.N_obs):
    print(t)
    statistics_etkf.propagate(25)
    etkFilter.filter(statistics_etkf.ensemble.ensemble, observation.obses[t])
    statistics_etkf.plot()

## LETKF filtering

Updating/conditioning the distributions at every observation time

In [ ]:
statistics_letkf = Statistics.Statistics(simulator, 100)
statistics_letkf.set_prior(prior_args)

statistics_letkf.plot()

In [ ]:
# We choose the scale_r as about 2/3 of the correlation length in grid cells 
scale_r = 8

In [ ]:
sletkFilter = SLETKalmanFilter.SLETKalman(statistics_letkf, observation, scale_r)

In [ ]:
sletkFilter.groups

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(sletkFilter.W_loc, origin="lower", vmin=0.0, vmax=1.0)
plt.colorbar()
plt.show()
for g in range(len(sletkFilter.groups)):
    plt.imshow(sletkFilter.W_analyses[g], origin="lower", vmin=0.0, vmax=1.0)
    plt.colorbar()
    plt.show()

In [ ]:
for t in range(observation.N_obs):
    print(t)
    statistics_letkf.propagate(25)
    sletkFilter.filter(statistics_letkf.ensemble.ensemble, observation.obses[t])
    statistics_letkf.plot()

## IEWPF filtering

In [ ]:
statistics_iewpf = Statistics.Statistics(simulator, 100)
statistics_iewpf.set_prior(prior_args)

statistics_iewpf.plot()

In [ ]:
iewpFilter = IEWParticleFilter.IEWParticle(statistics_iewpf, observation)

for t in range(observation.N_obs):
    print(t)
    statistics_iewpf.propagate(25)
    iewpFilter.filter(statistics_iewpf.ensemble.ensemble, observation.obses[t])
    statistics_iewpf.plot()

## Comparison

In [ ]:
from importlib import reload  
reload(Comparer)

In [ ]:
comparer = Comparer.Comparer(statistics_kf, statistics_etkf, statistics_letkf, statistics_iewpf)

In [ ]:
comparer.mean_plots()

In [ ]:
mean_rmse_kf, mean_rmse_etkf, mean_rmse_letkf, mean_rmse_iewpf = comparer.mean_rmse()

print("RMSE of KF = ", mean_rmse_kf)
print("RMSE of ETKF = ", mean_rmse_etkf)
print("RMSE of LETKF = ", mean_rmse_letkf)
print("RMSE of IEWPF = ", mean_rmse_iewpf)

In [ ]:
comparer.cov_plots()

In [ ]:
cov_frob_kf, cov_frob_etkf, cov_frob_letkf, cov_frob_iewpf = comparer.cov_frobenius_dist()

print("Frobenius error in Cov matrix (KF) = ", cov_frob_kf)
print("Frobenius error in Cov matrix (ETKF) = ", cov_frob_etkf)
print("Frobenius error in Cov matrix (LETKF) = ", cov_frob_letkf)
print("Frobenius error in Cov matrix (IEWPF) = ", cov_frob_iewpf)

In [ ]:
comparer.stddev_plots()

In [ ]:
stddev_rmse_kf, stddev_rmse_etkf, stddev_rmse_letkf, stddev_rmse_iewpf = comparer.stddev_rmse()

print("RMSE of KF = ", stddev_rmse_kf)
print("RMSE of ETKF = ", stddev_rmse_etkf)
print("RMSE of LETKF = ", stddev_rmse_letkf)
print("RMSE of IEWPF = ", stddev_rmse_iewpf)

In [ ]:
reload(Comparer)
comparer = Comparer.Comparer(statistics_kf, statistics_etkf, statistics_letkf, statistics_iewpf)

In [ ]:
comparer.set_poi([0,0])
comparer.set_poi([25,15])
comparer.set_poi([1,0])

In [ ]:
comparer.poi_plot(observation)

In [ ]:
comparer.poi_hist(0)
comparer.poi_hist(1)
comparer.poi_hist(2)

In [ ]:
comparer.poi_ecdf_plots(0)
comparer.poi_ecdf_plots(1)
comparer.poi_ecdf_plots(2)

In [ ]:
ecdf_err_etkf, ecdf_err_letkf, ecdf_err_iewpf = comparer.poi_ecdf_err(0)

print("Difference at pos0 (ETKF) = ", ecdf_err_etkf)
print("Difference at pos0 (LETKF) = ",ecdf_err_letkf)
print("Difference at pos0 (IEWPF) = ",ecdf_err_iewpf)

ecdf_err_etkf, ecdf_err_letkf, ecdf_err_iewpf = comparer.poi_ecdf_err(1)

print("Difference at pos1 (ETKF) = ", ecdf_err_etkf)
print("Difference at pos1 (LETKF) = ",ecdf_err_letkf)
print("Difference at pos1 (IEWPF) = ",ecdf_err_iewpf)

ecdf_err_etkf, ecdf_err_letkf, ecdf_err_iewpf = comparer.poi_ecdf_err(2)

print("Difference at pos2 (ETKF) = ", ecdf_err_etkf)
print("Difference at pos2 (LETKF) = ",ecdf_err_letkf)
print("Difference at pos2 (IEWPF) = ",ecdf_err_iewpf)